See how to use Shor's algorithm to factor 15 here:    <b><a href="https://portal.quantumrings.com/doc/Shors.html">Shor15</a></b>

<i><b>Source code to factorize 15</b></i>

Note: Be sure to use your API token and your account name.

Step 1. Import the required modules and obtain the backend

In [2]:
#!pip install qiskit[visualization]

In [3]:
import qiskit
print (qiskit.__version__)

1.3.2


In [5]:
#!pip install QuantumRingsLib

In [6]:
!python --version

Python 3.11.9


In [7]:
import QuantumRingsLib
from QuantumRingsLib import QuantumRegister, AncillaRegister, ClassicalRegister, QuantumCircuit
from QuantumRingsLib import QuantumRingsProvider
from QuantumRingsLib import job_monitor
from QuantumRingsLib import JobStatus
from matplotlib import pyplot as plt
import numpy as np
import math

provider = QuantumRingsProvider(
    token='rings-128.TdVKnV6k5pEiSIyMIRYG05gLXKmUOJkq',
    name='g.scorpiosky@gmail.com'
)
backend = provider.get_backend("scarlet_quantum_rings")
shots = 1024

provider.active_account()

{'name': 'g.scorpiosky@gmail.com',
 'token': 'rings-128.TdVKnV6k5pEiSIyMIRYG05gLXKmUOJkq',
 'max_qubits': '128'}

In [8]:
# qiskit lib
from qiskit.circuit.library import QFT

In [40]:
def phase_estimation(
        controlled_operation: QuantumCircuit,
        psi_prep: QuantumCircuit,
        precision: int
    ):
    """
    Carry out phase estimation on a simulator.
    Args:
        controlled_operation: The operation to perform phase estimation on,
                              controlled by one qubit.
        psi_prep: Circuit to prepare |ψ>
        precision: Number of counting qubits to use
    Returns:
        float: Best guess for phase of U|ψ>
    """
    control_register = QuantumRegister(precision)
    output_register = ClassicalRegister(precision)

    target_register = QuantumRegister(psi_prep.num_qubits)
    qc = QuantumCircuit(control_register, target_register, output_register)

    # Prepare |ψ>
    qc.compose(psi_prep,
               qubits=target_register,
               inplace=True)

    # Do phase estimation
    for index, qubit in enumerate(control_register):
        qc.h(qubit)
        for _ in range(2**index):
            qc.compose(
                controlled_operation,
                qubits=[qubit] + list(target_register),
                inplace=True,
            )

    qc.compose(
        QFT(precision, inverse=True),
        qubits=control_register,
        inplace=True
    )

    qc.measure(control_register, output_register)
    
    job = backend.run(qc, shots=shots)
    job_monitor(job)
    result = job.result()
    counts = result.get_counts()
    maxFreq = max(counts.values())
    
    phase = int(maxFreq/(2**N_COUNT))
    print(f"Corresponding Phase: {phase}")
    return phase

In [13]:
def a2jmodN(a, j, N):
    """Compute a^{2^j} (mod N) by repeated squaring"""
    for _ in range(j):
        a = np.mod(a**2, N)
    return a

In [54]:
def c_amod15(a,N):
    """
    Controlled multiplication by a mod 15 using QFT instead of SWAP.
    """
    Q = 2**ceil(log2(N))
    
    U = QuantumCircuit(Q)
    U.append(QFT(Q, do_swaps=False), range(Q))  # Apply QFT
    
    for qubit in range(Q):
        U.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            U.cp(-np.pi/float(2**(j-m)), m, j)
        U.h(j)
    
    U.append(qft_dagger(Q), range(Q))  # Apply inverse QFT
    
    U = U.to_gate()
    U.name = f"{a} mod N"
    c_U = U.control()
    return c_U
                

In [55]:
from fractions import Fraction
from math import gcd, ceil, log2
j = 143
N = 8
a = a2jmodN(a, j, N)
N = 15

Q = 2**ceil(log2(N))
psi_prep = QuantumCircuit(Q)
psi_prep.x(0)

FACTOR_FOUND = False
ATTEMPT = 0
while not FACTOR_FOUND:
    ATTEMPT += 1
    print(f"\nAttempt {ATTEMPT}")

    phase = phase_estimation(
        c_amod15(a,N),
        psi_prep,
        precision=Q
    )
    frac = Fraction(phase).limit_denominator(N)
    r = frac.denominator
    if phase != 0:
        # Guess for a factor is gcd(x^{r/2} - 1 , 15)
        guess = gcd(a ** (r // 2) - 1, N)
        if guess not in [1, N] and (N % guess) == 0:
            # Guess is a factor!
            print(f"Non-trivial factor found: {guess}")
            FACTOR_FOUND = True


Attempt 1


TypeError: append(): incompatible function arguments. The following argument types are supported:
    1. (self: QuantumRingsLib.QuantumCircuit, qc: QuantumRingsLib.QuantumCircuit, qubits_vector: list[int], cbits_vector: list[int]) -> int
    2. (self: QuantumRingsLib.QuantumCircuit, g: QuantumRingsLib.Gate, qubits_vector: list[int], cbits_vector: list[int]) -> int

Invoked with:          
q[0]:  ■ 
         
q[1]:  ■ 
         
q[2]:  ■ 
         
q[3]:  ■ 
         
q[4]:  ■ 
         
q[5]:  ■ 
         
q[6]:  ■ 
         
q[7]:  ■ 
         
q[8]:  ■ 
         
q[9]:  ■ 
         
q[10]: ■ 
         
q[11]: ■ 
         
q[12]: ■ 
         
q[13]: ■ 
         
q[14]: ■ 
         
q[15]: ■ 
         
c: 16/ ■ 
         
, <qiskit.circuit.library.basis_change.qft.QFT object at 0x7acf162ea310>, range(0, 16)

Step 2. Define the core methods

In [ ]:
def iqft_cct(qc, b, n):
    """
    The inverse QFT circuit

    Args:

        qc (QuantumCircuit):
                The quantum circuit

        b (QuantumRegister):
                The target register

        n (int):
                The number of qubits in the registers to use

    Returns:
        None

    """

    for i in range (n):
        for j in range (1, i+1):
            # for inverse transform, we have to use negative angles
            qc.cu1(  -math.pi / 2** ( i -j + 1 ), b[j - 1], b[i])
        # the H transform should be done after the rotations
        qc.h(b[i])
    qc.barrier()
    return

def plot_histogram (counts, title=""):
    """
    Plots the histogram of the counts

    Args:

        counts (dict):
            The dictionary containing the counts of states

        titles (str):
            A title for the graph.

    Returns:
        None

    """
    fig, ax = plt.subplots(figsize =(10, 7))
    plt.xlabel("States")
    plt.ylabel("Counts")
    mylist = [key for key, val in counts.items() for _ in range(val)]

    unique, inverse = np.unique(mylist, return_inverse=True)
    bin_counts = np.bincount(inverse)

    plt.bar(unique, bin_counts)

    maxFreq = max(counts.values())
    plt.ylim(ymax=np.ceil(maxFreq / 10) * 10 if maxFreq % 10 else maxFreq + 10)
    # Show plot
    plt.title(title)
    plt.show()
    return

Step 3. Perform the algorithm

In [ ]:
# Shor’s algorithm to factorize 15 using 7^x mod 15.
numberofqubits = 7
shots = 1024

q = QuantumRegister(numberofqubits , 'q')
c = ClassicalRegister(4 , 'c')
qc = QuantumCircuit(q, c)

# Initialize source and target registers
qc.h(0)
qc.h(1)
qc.h(2)
qc.x(6)
qc.barrier()

# Modular exponentiation 7^x mod 15
qc.cx(q[2],q[4] )
qc.cx(q[2],q[5] )
qc.cx(q[6],q[4] )
qc.ccx(q[1],q[5],q[3] )
qc.cx(q[3],q[5] )
qc.ccx(q[1],q[4],q[6] )
qc.cx(q[6],q[4] ) #
qc.barrier()

# IQFT. Refer to implementation from earlier examples
iqft_cct (qc, q, 3)

# Measure
qc.measure(q[0], c[0])
qc.measure(q[1], c[1])
qc.measure(q[2], c[2])

# Draw the circuit
qc.draw('mpl')

The circuit to factor 15 shown above.

In [ ]:
# Execute the circuit
job = backend.run(qc, shots=shots)
job_monitor(job)
result = job.result()
counts = result.get_counts()

# visualize
plot_histogram(counts)

#clean up
del q, c, qc
del result
del job

A plot of the execution results is shown above. Compare this with the calculated values.

Footnotes

[1] This section is based on [10], [14], and [16].

[2] https://research.ibm.com/blog/factor-15-shors-algorithm

[3] https://en.wikipedia.org/wiki/Integer_factorization_records#Records_for_efforts_by_quantum_computers